In [243]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import finufft
import time
from utils.kernels import get_xis
from kernels.squared_exponential import SquaredExponential
import math
from torch.profiler import profile, record_function, ProfilerActivity
import pytorch_finufft.functional as pff
from typing import Dict, Optional, Callable, Tuple
from cg import ConjugateGradients, BatchConjugateGradients
from typing import Optional, Tuple, Dict
import pytorch_finufft.functional as pff
from cg import ConjugateGradients, BatchConjugateGradients
from efgpnd import ToeplitzND, compute_convolution_vector_vectorized_dD, _cmplx, nufft_var_est_nd, diag_sums_nd, efgp_nd, EFGPND, efgpnd_gradient_batched
import warnings
warnings.filterwarnings("ignore", message=".*disabling cuda.*")


In [244]:


# # --- Parameters ---
# n = 5_000  # Number of points
# d = 2  # Dimensionality of the input space
# true_length_scale =0.5
# true_variance = 0.1
# true_noise_variance = 0.2
# dtype = torch.float64  # Use float64 as in the original example
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available
# print(f"Using device: {device}")

# # --- Generate Input Points ---
# # Generate random points in d-dimensional space from -1 to 1
# x = torch.rand(n, d, dtype=dtype, device=device) * 2 - 1

# # Generate test points (grid of points for visualization if d <= 3)
# if d >= 2:
#     # Create a grid of points for testing
#     grid_points_per_dim = 10
#     grid_points = [torch.linspace(x.min()-0.1, 0.1+x.max(), grid_points_per_dim, dtype=dtype, device=device) for _ in range(d)]
#     mesh_grid = torch.meshgrid(*grid_points, indexing='ij')
#     x_new = torch.stack([grid.flatten() for grid in mesh_grid], dim=1)
# elif d==1:
#     grid_points_per_dim = 3000
#     grid_points = [torch.linspace(x.min()-0.1, 0.1+x.max(), grid_points_per_dim, dtype=dtype, device=device) for _ in range(d)]
#     mesh_grid = torch.meshgrid(*grid_points, indexing='ij')
#     x_new = torch.stack([grid.flatten() for grid in mesh_grid], dim=1)
# else:
#     # For higher dimensions, just use random test points
#     x_new = torch.rand(1000, d, dtype=dtype, device=device) * 2.4 - 1.2

# # --- Define Mean and Covariance Functions (using PyTorch) ---
# def mean_func_torch(x):
#     """Mean function (returns zero vector)."""
#     # Make sure output shape matches input batch shape if any
#     return torch.zeros(x.shape[0], dtype=x.dtype, device=x.device)

# # Squared Exponential Kernel Function for d-dimensional inputs
# def squared_exponential_kernel(x1, x2, length_scale=1.0, variance=1.0):
#     """
#     Squared Exponential kernel using PyTorch.
#     Calculates variance * exp(-0.5 * ||x1 - x2||^2 / length_scale^2)
#     Handles multi-dimensional (n, d) inputs.
#     """
#     # Compute pairwise squared Euclidean distances
#     # dist_sq = ||a - b||^2 = ||a||^2 - 2a^T b + ||b||^2
#     sum_sq_x1 = torch.sum(x1**2, dim=1, keepdim=True)  # Shape (n, 1)
#     sum_sq_x2 = torch.sum(x2**2, dim=1)                # Shape (m,)
#     cross_term = torch.matmul(x1, x2.T)                # Shape (n, m)

#     # sq_dist shape: (n, 1) + (m,) - 2 * (n, m) -> broadcasting -> (n, m)
#     sq_dist = sum_sq_x1 + sum_sq_x2 - 2 * cross_term

#     # Ensure squared distances are non-negative (due to potential floating point inaccuracies)
#     sq_dist = torch.clamp(sq_dist, min=0.0)

#     # Compute the Squared Exponential kernel
#     cov = variance * torch.exp(-0.5 * sq_dist / length_scale**2)
#     return cov

# # --- Optimized Sampling Function ---
# def sample_gp_fast(x, mean_func, cov_func, num_samples, length_scale, variance, noise_variance):
#     """
#     Samples efficiently from a d-dimensional Gaussian Process by pre-computing
#     the Cholesky decomposition of the noisy covariance matrix.

#     Parameters:
#     x : torch.Tensor (n, d)
#         Input points in d-dimensional space.
#     mean_func : callable
#         Mean function.
#     cov_func : callable
#         Covariance function (kernel).
#     num_samples : int
#         Number of samples to draw.
#     length_scale : float
#         Kernel length scale.
#     variance : float
#         Kernel variance.
#     noise_variance : float
#         Variance of the observation noise.

#     Returns:
#     torch.Tensor (n, num_samples)
#         Samples from the GP at the input points x.
#     """
#     n_points = x.shape[0]
#     mean = mean_func(x)  # Shape (n_points,)

#     # 1. Compute Covariance Matrix K
#     K = cov_func(x, x, length_scale=length_scale, variance=variance)  # (n_points, n_points)

#     # 2. Add Noise Variance to Diagonal
#     K_noisy = K + noise_variance * torch.eye(n_points, dtype=x.dtype, device=x.device)

#     # 3. Compute Cholesky Factor L (K_noisy = L L^T)
#     try:
#         L = torch.linalg.cholesky(K_noisy)  # (n_points, n_points), lower triangular
#     except torch.linalg.LinAlgError as e:
#         print(f"Cholesky decomposition failed: {e}")
#         # Fallback with jitter
#         jitter = 1e-6 * torch.eye(n_points, dtype=x.dtype, device=x.device)
#         try:
#             L = torch.linalg.cholesky(K_noisy + jitter)
#             print("Cholesky succeeded after adding jitter.")
#         except torch.linalg.LinAlgError as e2:
#             print(f"Cholesky decomposition failed even with jitter: {e2}")
#             raise RuntimeError("Could not compute Cholesky decomposition.") from e2

#     # 4. Efficient Sampling
#     # Draw standard normal samples Z ~ N(0, I)
#     Z = torch.randn(n_points, num_samples, dtype=x.dtype, device=x.device)  # (n_points, num_samples)

#     # Compute samples: Y = mean + L @ Z
#     samples = mean.unsqueeze(1) + L @ Z  # (n_points, num_samples)

#     return samples

# # --- Generate Samples ---
# print(f"Generating {1} sample for n={n} points in {d} dimensions...")
# start_time = time.time()

# # Sample using the optimized function
# y = sample_gp_fast(
#     x,
#     mean_func_torch,
#     squared_exponential_kernel,
#     num_samples=1,
#     length_scale=true_length_scale,
#     variance=true_variance,
#     noise_variance=true_noise_variance
# ).flatten()

# end_time = time.time()
# print(f"Generated sample of size {y.shape[0]}.")
# print(f"Time taken: {end_time - start_time:.4f} seconds")


In [245]:
data = torch.load("/Users/colecitrenbaum/Documents/GPs/gp-quadrature/gp_samples_100_0.5_2_0.2.pt")
x = data[0]
y = data[1]


In [246]:
x.shape

torch.Size([100, 2])

In [247]:
d= 2 
kernel = SquaredExponential(dimension=d, lengthscale=torch.tensor([0.23]), variance=1)
init_kernel = kernel
sigmasq = torch.tensor(0.1)
EPSILON = 1e-2

In [248]:
cg_tol = 1e-4
J = 500

model = EFGPND(
    x        = x,               # training inputs
    y        = y,               # training targets
    kernel   = kernel,
    sigmasq  = sigmasq,
    eps      = EPSILON,
    opts     = {
        "cg_tolerance"     : cg_tol,
        "early_stopping"   : True,
    },
)

# # Solve once
# model.fit()

# # Predict at test points
# mean, var = model.predict(
#     x_new,
#     return_variance   = True,
#     variance_method   = "regular",
# )



In [249]:
x.shape

torch.Size([100, 2])

In [250]:
kernel

SquaredExponential(dimension=2, lengthscale=0.23000000417232513, variance=1.0)

In [251]:
x0 = x.min(dim=0).values  
x1 = x.max(dim=0).values  
# set seed
# torch.manual_seed(42)
# from efgpnd import efgpnd_gradient_batched as efgpnd_gradient_batched_2
# efgpnd_gradient_batched_2(x, y, sigmasq, kernel, EPSILON, trace_samples=10, x0=x0, x1=x1)

tensor([-6.9821e+01,  1.3798e-01, -2.8031e+02], dtype=torch.float64)

In [252]:
kernel

SquaredExponential(dimension=2, lengthscale=0.23000000417232513, variance=1.0)

In [ ]:
torch.manual_seed(42)
model = EFGPND(x, y, kernel, sigmasq, EPSILON).optimize_hyperparameters(
    epsilon_values       = [1e-2],
    trace_samples_values = [10],
    lr                   = 0.01,
    max_iters            = 50,
    x0 = x0, x1 = x1,                    
)

# now, training_log contains per‑iteration traces
fit = model.fit()                              # build posterior with new θ
print("final hyperparams",model.kernel.lengthscale,model.kernel.variance,model.sigmasq)



/Users/colecitrenbaum/Documents/GPs/gp-quadrature/efgpnd.py:477: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sigmasq = torch.tensor(self.sigmasq, dtype=rdtype, device=device)


[ε=0.01 | J=10] iter   0  ℓ=0.2701  σ_f²=0.9986  σ_n²=0.1324
grad: tensor([-6.9821e+01,  1.3798e-01, -2.8031e+02], dtype=torch.float64)
[ε=0.01 | J=10] iter  10  ℓ=0.4779  σ_f²=1.104  σ_n²=0.237
grad: tensor([-11.9848,  -0.4948,  -2.7350], dtype=torch.float64)
[ε=0.01 | J=10] iter  20  ℓ=0.4935  σ_f²=1.217  σ_n²=0.2388
grad: tensor([-14.7122,   0.6356,  -4.6734], dtype=torch.float64)
[ε=0.01 | J=10] iter  30  ℓ=0.5034  σ_f²=1.27  σ_n²=0.2422
grad: tensor([-6.3654,  0.8788, -9.3623], dtype=torch.float64)
[ε=0.01 | J=10] iter  40  ℓ=0.4884  σ_f²=1.357  σ_n²=0.2358
grad: tensor([ 1.9748, -0.4962, -2.9148], dtype=torch.float64)
[ε=0.01 | J=10] iter  50  ℓ=0.4893  σ_f²=1.337  σ_n²=0.2403
grad: tensor([ 9.7006, -1.1636, -0.8992], dtype=torch.float64)
[ε=0.01 | J=10] iter  60  ℓ=0.4737  σ_f²=1.343  σ_n²=0.2377
grad: tensor([ 8.6380, -0.6319, -0.5638], dtype=torch.float64)
[ε=0.01 | J=10] iter  70  ℓ=0.4782  σ_f²=1.352  σ_n²=0.2402
grad: tensor([ 5.5305, -0.0351, -1.0377], dtype=torch.float64)

In [254]:
model.kernel

SquaredExponential(dimension=2, lengthscale=0.5072881894108254, variance=1.383278993232624)

In [255]:
print("final hyperparams",model.lengthscale)


AttributeError: 'EFGPND' object has no attribute 'lengthscale'

In [43]:
# fit.training_log

In [ ]:
# add plot of fit and true function

if d == 2:

    import plotly.graph_objects as go
    import numpy as np
    from scipy.interpolate import griddata

    # --- Prepare Data for Plotting ---
    # Extract x1, x2 coordinates and mean values from tensors
    x1 = x_new[:, 0].cpu().numpy()
    x2 = x_new[:, 1].cpu().numpy()
    mean_values = mean.cpu().numpy()

    # Create a grid for the surface plot
    x1_grid = np.linspace(x1.min(), x1.max(), 100)
    x2_grid = np.linspace(x2.min(), x2.max(), 100)
    X1, X2 = np.meshgrid(x1_grid, x2_grid)

    # Interpolate mean values onto the grid for smooth surface visualization
    mean_surface = griddata(
        points=(x1, x2), 
        values=mean_values, 
        xi=(X1, X2), 
        method='cubic'
    )

    # --- Create Interactive 3D Plot ---
    fig = go.Figure()

    # Add the mean surface with transparency
    fig.add_trace(go.Surface(
        z=mean_surface,
        x=X1,
        y=X2,
        colorscale='Viridis',
        opacity=0.8,  # Adjust transparency for better visualization
        name='EFGP Mean',
        colorbar=dict(
            title=dict(
                text="Mean Value",
                font=dict(size=14)
            ),
            tickfont=dict(size=12)
        )
    ))

    # Add scatter points for the true data
    fig.add_trace(go.Scatter3d(
        x=x[:, 0].cpu().numpy(),
        y=x[:, 1].cpu().numpy(),
        z=y.cpu().numpy(),
        mode='markers',
        marker=dict(
            size=4,  # Smaller scatter points
            color='red',
            opacity=0.9,  # Slight transparency for markers
            symbol='circle'
        ),
        name='True Data'
    ))

    # --- Update Layout for Better Visualization ---
    fig.update_layout(
        scene=dict(
            aspectmode='manual',
            aspectratio=dict(x=1.5, y=1.5, z=1), # Make x and y dimensions wider relative to z
            xaxis=dict(
                title=dict(
                    text='x₁',
                    font=dict(size=16)
                ),
                tickfont=dict(size=12),
                showgrid=True,
                gridcolor='lightgray'
            ),
            yaxis=dict(
                title=dict(
                    text='x₂',
                    font=dict(size=16)
                ),
                tickfont=dict(size=12),
                showgrid=True,
                gridcolor='lightgray'
            ),
            zaxis=dict(
                title=dict(
                    text='Mean/True Data',
                    font=dict(size=16)
                ),
                tickfont=dict(size=12),
                showgrid=True,
                gridcolor='lightgray'
            ),
            camera=dict(
                eye=dict(x=1.25, y=1.25, z=1)  # Adjust camera angle for better view
            )
        ),
        title=dict(
            text='EFGP Mean and True Data',
            font=dict(size=18),
            x=0.5,  # Center the title
            xanchor='center'
        ),
        legend=dict(
            x=0.8,
            y=0.9,
            font=dict(size=12),
            bgcolor='rgba(255, 255, 255, 0.8)',  # Add a semi-transparent background
            bordercolor='black',
            borderwidth=1
        ),
        margin=dict(l=0, r=0, t=50, b=0),  # Reduce margins for a cleaner look
        width=800,  # Increase figure width
        height=600   # Increase figure height
    )

    # Display the plot
    fig.show()


